第一个算法是暴力穷举法
The first algorithms we looked at in this book were based on brute-force exhaustive enumeration.

一般来说，比较好的策略是先用最简单直接的方式解决手头的 问题，再仔细测试找出计算上的瓶颈，然后仔细研究造成瓶颈的那部分程序，并找出改善计算复 杂度的方法。
It is often a good strategy to start by solving the problem at hand in the most straightforward manner possible, instrument it to find any computational bottlenecks, and then look for ways to improve the computational complexity of those parts of the program contributing to the bottlenecks.
# 10.1 搜索算法 Search Algorithms
搜索算法就是在一个项目集合中找出一个或一组具有某种特点的项目。我们将项目集合称为 搜索空间。
A search algorithm is a method for finding an item or group of items with specific properties within a collection of items. We refer to the collection of items as a search space.
这个例子演示了计算中最重要的实现技术之一： 间接引用。
This example illustrates one of the most important implementation techniques used in computing: indirection.
常有人说： “计算中的任何问题都可以通过添加另一个间接层来解决。”
It has often been said that “any problem in computing can be solved by adding another level of indirection.”
我们每 次使用变量引用与变量绑定的对象时，就是这么做的。当我们使用一个变量访问列表并使用保存 在列表中的引用访问另一个对象时，实际上进行了双重间接引用。
. When we use a variable to access a list and then a reference stored in that list to access another object, we are going through two levels of indirection
使用二分查找时，我们依赖的是浮点数固有的全序 性。
We can, however, get a considerable improvement in the worst-case complexity by using an algorithm, binary search, that is similar to the bisection search algorithm used in Chapter 3 to find an approximation to the square root of a floating point number. There we relied upon the fact that there is an intrinsic total ordering on floating point numbers.
实现二分查找的最简单直接的方式就是使用递归，
it is not surprising that the most straightforward implementation of binary search uses recursion,




In [1]:
def search(L,e):
    def bSearch(L,e,low,high):
        if high == low:
            return L[low]==e
        mid = (low + high)//2
        if L[mid] == e:
            return True
        elif L[mid] > e:
            if low == mid:
                return False
            else:
                return bSearch(L,e, low, mid-1)
        else:
            return bSearch(L,e,mid + 1, high)
    if len(L) == 0:
        return False
    else:
        return bSearch(L,e, 0, len(L)-1)



[1, 2, 3, 5, 4] [5, 3, 4, 2, 1]


像search这样的函数经常被称为包装器函数。这种函数为客户代码提供了一个非常易用的接 口，但就是一个外壳，不执行重要的计算，而使用适当的参数调用辅助函数bSearch。这就引起 一个问题：为什么不去掉search，让客户代码直接调用bSearch呢？原因就在于bSearch中的两 个参数low和high，它们与在列表中搜索一个元素这一抽象任务没有任何关系，只是具体实现中 的细节，应该对search的调用者隐藏
Functions such as search are often called wrapper functions. The function provides a nice interface for client code, but is essentially a pass-through that does no serious computation. Instead, it calls the helper function bSearch with appropriate arguments. This raises the question of why not eliminate search and have clients call bSearch directly? The reason is that the parameters low and high have nothing to do with the abstraction of searching a list for an element. They are implementation details that should be hidden from those writing programs that call search

# 10.2 排序算法 Sorting Algorithms
从上一节可以看到，如果知道列表是有序的，那么我们就可以利用这个信息大大降低搜索列 表所需的时间。这是否意味着在有列表搜索的需求时，应该先排序再执行搜索呢？
We have just seen that if we happen to know that a list is sorted, we can exploit that information to greatly reduce the time needed to search a list. Does this mean that when asked to search a list one should first sort it and then perform the search?
我们应该使用Python内置的sort方法（ L.sort()可以对列表L排序），或者使用内置 函数sorted（ sorted(L)会返回一个列表，其中包含与L同样的元素，但是不会修改L）
 In practice, you will rarely need to implement your own sort function. In most cases, the right thing to do is to use either Python’s built-in sort method (L.sort() sorts the list L) or its built-in function sorted (sorted(L) returns a list with the same elements as L, but does not mutate L)

 > 循环不变式loop Invariant

选择排序的工作原理是 维持一个循环不变式，它会将列表分成前缀部分（ L[0 : i]）和后缀部分（ L[i+1 : len(L)]）， 前缀部分已经排好序，而且其中的每一个元素都不大于后缀部分中的最小元素
 Selection sort, Figure 10.4, works by maintaining the loop invariant that, given a partitioning of the list into a prefix (L[0:i]) and a suffix (L[i+1:len(L)]), the prefix is sorted and no element in the prefix is larger than the smallest element in the suffix.

>推导 reason

我们使用归纳法对循环不变式进行推导。
We use induction to reason about loop invariants
- 基础情形：第一次迭代开始时，前缀集合是空的，也就是说，后缀集合是整个列表。因 此，不变式（显然）成立。
- 归纳步骤：在算法的每一步中，我们都从后缀集合向前缀集合移动一个元素，移动的方 式是将后缀集合中的最小元素添加到前缀集合的末尾。因为移动元素之前，不变式是成 立的，所以添加元素之后，前缀集合依然有序。而且，因为我们从后缀集合中移走的是 最小元素，所以前缀集合中仍然没有任何一个元素大于后缀集合中的最小元素。
- 结束：退出循环时，前缀集合中包括了整个列表，后缀集合是空的。因此，整个列表按 照升序排列。
1. Base case: At the start of the first iteration, the prefix is empty, i.e., the suffix is the entire list. Therefore, the invariant is (trivially) true.
2. Induction step: At each step of the algorithm, we move one element from the suffix to the prefix. We do this by appending a minimum element of the suffix to the end of the prefix. Because the invariant held before we moved the element, we know that after we append the element the prefix is still sorted. We also know that since we removed the smallest element in the suffix, no element in the prefix is larger than the smallest element in the suffix.
3. Termination: When the loop is exited, the prefix includes the entire list, and the suffix is empty. Therefore, the entire list is now sorted in ascending order

幸运的是，我们可以使用分治算法得到比平方复杂度好得多的结果。其基本思想就是先找出 初始问题的一些简单实例的解，再将这些解组合起来作为初始问题的解。
Fortunately, we can do a lot better than quadratic time using a divide-and-conquer algorithm. The basic idea is to combine solutions of simpler instances of the original problem. In general, a divide-and-conquer algorithm is characterized by
阈值有时被称为递归基
The threshold is sometimes called the recursive base

归并排序是一种典型的分治算法，它由约翰·冯·诺依曼于1945年发明，至今仍被广泛使用。
和多数分治算法一样，用递归方式描述它是最容易的：
Merge sort is a prototypical divide-and-conquer algorithm. It was invented in 1945, by John von Neumann, and is still widely used. Like many divide-and-conquer algorithms it is most easily described recursively:
(1) 如果列表的长度是0或1，那么它已经排好序了；
(2) 如果列表包含多于1个元素，就将其分成两个列表，并分别使用归并排序法进行排序；
(3) 合并结果。
1. If the list is of length 0 or 1, it is already sorted.
2. If the list has more than one element, split the list into
two lists, and use merge sort to sort each of them.
3. Merge the results.
冯·诺依曼的关键发现是，两个有序的列表可以高效地合并成一个有序列表。
The key observation made by von Neumann is that two sorted lists can be efficiently merged into a single sorted list.


In [ ]:
def merge(left, right, compare):
    result = []
    i,j = 0,0
    while i < len(left) and j < len(right):
        if compare(left[i], right[i]):
            result.append(left[i])
            i +=1
        else:
            result.append(right[j])
            j += 1
    while (i<len(left)):
        result.append(left[i])
        i+=1
    while (j<len(right)):
        result.append(right[j])
        j += 1
    return result

def mergeSort(L, compare = lambda x, y: x<y):
    if len(L)<2:
        return L[:]
    else:
        middle = len(L)//2
        left = mergeSort(L[:middle],compare)
        right = mergeSort(L[middle:],compare)
        return merge(left, right, compare)

L = [2,1,4,5,3]
print(mergeSort(L), mergeSort(L, lambda x,y:x>y))


多数Python版本中使用的排序算法被称为timsort。这种算法的核心思想是利用这样一个事 实，即在很多数据集中，数据已经部分有序
The sorting algorithm used in most Python implementations is called timsort.64 The key idea is to take advantage of the fact that in a lot of data sets the data is already partially sorted.Timsort’s worst-case performance is the same as merge sort’s, but on average it performs considerably better.
# 散列表 Hash Table
 散列表背后的基本思想非常简单，我们将键转换为一个整数，然后 使用这个整数索引一个列表，这都可以在常数时间内完成。
  The basic idea behind a hash table is simple. We convert the key to an integer, and then use that integer to index into a list, which can be done in constant time.
 为了解决这个问题，我们引入散列函数。它会将一个大规模的输入空间（如所有自然数）映 射为一个小规模的输出空间（如0~5000的自然数）。
A hash function maps a large space of inputs (e.g., all natural numbers) to a smaller space of outputs (e.g., the natural
numbers between 0 and 5000).
因为输出空间小于输入空间，所以散列函数是个多对一映射。也就是说，多个不同输入会被 映射为同一输出。当两个输入被映射为同一个输出时，我们称这种情况为碰撞，随后会对其进行 介绍。一个好的散列函数会生成一个均匀分布，也就是说，范围内出现每种输出的可能性都是相 等的，这会使产生碰撞的可能性最小化
Since the space of possible outputs is smaller than the space of possible inputs, a hash function is a many-to-one mapping, i.e., multiple different inputs may be mapped to the same output. When two inputs are mapped to the same output, it is called a collision—a topic we will return to shortly. A good hash function produces a uniform distribution; i.e., every output in the range is equally